**NAIVE BAYES**

Baseado no teorema de bayes (análise probabilistica)

Aplicações mais comuns :

- Detecção de SPAN
- Detecção de emoções em frases
- Separação de documentos

O naive bayes a partir da tabela de dados previsores gera uma tabela de probabilidades que é usada como base para classificar novos dados.

Vantagens :

- Rápido se comparado a abordagens mais complexas ( Ex: redes neurais )
- Simples
- Capaz de tabalhar com altas dimensões (atributos)
- Boas previsões em bases de dados pequenas ( 400 - 1000 registros)

Desvantagens :
- Presume que os atributos previsores são totalmente independentes, o que nem sempre é verdade.



In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


# carregando a base de dados de  credito
base = pd.read_csv('../../res/credit-data.csv')
#base = pd.read_csv('../../res/risco-credito.csv')
base.shape


(2000, 5)

In [3]:
#efetuando correcoes nos dados de credito

print('Eliminando as idades negativas\n')

#em alguns registros o campo idade possui numeros menores que zero
regs =base.loc[base['age'] < 0] 
print('antes :\n')
print(base.iloc[regs.index])

# calculando a media dos valores nao negativos do campo idade
media = base[base['age']>0]['age'].mean()

#setando os valores negativos com o valor da media dos nao negativos
base.loc[base.age < 0, 'age'] = media

print('\ndepois :\n')
print(base.iloc[regs.index])

print('\nEliminando registros NaN da base\n')
print('antes :\n')


regs=base.loc[pd.isnull(base).any(axis=1)]
print(base.iloc[regs.index])

for column in base.columns:
    base[column].fillna(base[column].mean(),inplace=True)

print('\ndepois : \n')
print(base.iloc[regs.index])

Eliminando as idades negativas

antes :

    clientid        income        age         loan  default
15        16  50501.726689 -28.218361  3977.287432        0
21        22  32197.620701 -52.423280  4244.057136        0
26        27  63287.038908 -36.496976  9595.286289        0

depois :

    clientid        income      age         loan  default
15        16  50501.726689  40.9277  3977.287432        0
21        22  32197.620701  40.9277  4244.057136        0
26        27  63287.038908  40.9277  9595.286289        0

Eliminando registros NaN da base

antes :

    clientid        income  age         loan  default
28        29  59417.805406  NaN  2082.625938        0
30        31  48528.852796  NaN  6155.784670        0
31        32  23526.302555  NaN  2862.010139        0

depois : 

    clientid        income      age         loan  default
28        29  59417.805406  40.9277  2082.625938        0
30        31  48528.852796  40.9277  6155.784670        0
31        32  23526.302555  40

In [4]:
# separando previsores e classificadores
previsores = base.iloc[:, 1:4].values
classificadores = base.iloc[:, 4].values

print ( base.iloc[:, 4].unique())

# padronizando os valores dos previsores
scaler = StandardScaler()
previsores = scaler.fit_transform(previsores)

# separando os valores de teste e treinamento 
previsores_treinamento, previsores_teste, classificadores_treinamento, classificadores_teste = train_test_split(previsores, classificadores, test_size=0.25, random_state=0)

print ("previsores treinamento /teste ",previsores_treinamento.shape,previsores_teste.shape)
print ("classificadores treinamento /teste ",classificadores_treinamento.shape,classificadores_teste.shape)

[0 1]
previsores treinamento /teste  (1500, 3) (500, 3)
classificadores treinamento /teste  (1500,) (500,)


In [5]:
# instanciando o naive bayes com o scikit
classificador = GaussianNB()
classificador.fit(previsores_treinamento, classificadores_treinamento)

# rodando previsoes com o dado de teste
previsoes = classificador.predict(previsores_teste)

In [6]:
#testes dessa instancia algoritimo

# o dado de precisao per se nao quer dizer muita coisa e preciso verificar outras metricas
precisao = accuracy_score(classificadores_teste, previsoes)

# uma dessas metricas eh a matriz de confusao ... ela e capaz de mostrar o desempenho do algoritimo para cada classe 
matriz = confusion_matrix(classificadores_teste, previsoes)

#o scikit tambem possui uma classe utilitaria que prove um report mais detalhado...
report = classification_report(classificadores_teste, previsoes)

print("Precisão:\n")
print(precisao)
print("\nMatriz de confusão:\n")
print(matriz)
print("\nReport:\n")
print (report)

Precisão:

0.938

Matriz de confusão:

[[428   8]
 [ 23  41]]

Report:

              precision    recall  f1-score   support

           0       0.95      0.98      0.97       436
           1       0.84      0.64      0.73        64

    accuracy                           0.94       500
   macro avg       0.89      0.81      0.85       500
weighted avg       0.93      0.94      0.93       500

